# How to .... ?
A FAQ-like section to quickly find answers to common questions.

### ... get a `np.array` returned instead of a `xr.DataArray`?

There are two ways: First, use the global config to control the general behaviour by calling `h5tbx.config.return_xarray=False`. Second option is to call `.values[:]` on the dataset:

In [1]:
import h5rdmtoolbox as h5tbx

h5tbx.config.return_xarray = False
with h5tbx.H5File() as h5:
    h5.create_dataset('test', shape=(3, 4), units='', long_name='a long name')
    print(type(h5['test'][:]))
    h5tbx.config.return_xarray = True
    print(type(h5['test'][:]))
    h5tbx.config.return_xarray = True
    print(type(h5['test'].values[:]))

<class 'numpy.ndarray'>
<class 'xarray.core.dataarray.DataArray'>
<class 'numpy.ndarray'>


## ... avoid having to provide `units` and `long_name` or `standard_name` during dataset creation
This cannot be avoided as we believe these are minimal auxiliary information (meta data) required with every dataset. If unitless, pass `units=' '` and if you are not using any naming standard provide `long_name` which is just a description of your dataset and should be easy to think of.